In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

"""
Load ECoG data with MNE, speech consonant-vowels dataset, 256 channels [Bouchard & Chang 2019]
Authored by Eric Easthope

MIT License
Copyright (c) 2022
"""

# !pip install pynwb mne --quiet

import pynwb
import mne

# Load ECoG data with MNE
Authored by Eric Easthope

The "human ECoG speaking consonant-vowel syllables" dataset [Bouchard & Chang 2019] has a separate [NWB](https://www.nwb.org)-formatted file for each subject ID, available [here](https://figshare.com/collections/Human_ECoG_speaking_consonant-vowel_syllables/4617263). If we download one of these NWB files and point Python to where it is with `filename` we can load it with PyNWB and preview its contents.

In [2]:
# Set filepath
# Load NWB file
filepath = "data/ecog-cv-syllables/EC9_B53.nwb"
nwb = pynwb.NWBHDF5IO(filepath, "r").read()
nwb

root pynwb.file.NWBFile at 0x5140874528
Fields:
  acquisition: {
    ElectricalSeries <class 'pynwb.ecephys.ElectricalSeries'>
  }
  devices: {
    auto_device <class 'pynwb.device.Device'>
  }
  electrode_groups: {
    auto_group <class 'pynwb.ecephys.ElectrodeGroup'>
  }
  electrodes: electrodes <class 'hdmf.common.table.DynamicTable'>
  epochs: epochs <class 'pynwb.epoch.TimeIntervals'>
  file_create_date: [datetime.datetime(2019, 6, 19, 12, 29, 13, 358846, tzinfo=tzoffset(None, -25200))]
  identifier: EC9_B53
  institution: University of California, San Francisco
  intervals: {
    epochs <class 'pynwb.epoch.TimeIntervals'>,
    invalid_times <class 'pynwb.epoch.TimeIntervals'>,
    trials <class 'pynwb.epoch.TimeIntervals'>
  }
  invalid_times: invalid_times <class 'pynwb.epoch.TimeIntervals'>
  lab: Chang Lab
  session_description: EC9_B53
  session_id: EC9_B53
  session_start_time: 1900-01-01 08:00:00+00:00
  subject: subject pynwb.file.Subject at 0x5141206352
Fields:
  species:

Time series data for each subject is considered an `ElectricalSeries` object, which we can see is under `"acquisition"`. Though each NWB file can be quite large, so we may only wish to look at a few seconds of signal at a time. We can set this with `start` and `num_secs`, so that we get `num_secs` seconds of data after `start` seconds.

In [3]:
# Set starting time
# Set number of seconds to capture
start = 20
num_secs = 60

To get the time series data itself data we access `"ElectricalSeries"` as though the `"acquisition"` attribute is a Python dictionary, and see that each `ElectricalSeries` object has a `"data"` attibute. Here we also find the sampling frequency `fs` as a `"rate"` attribute, which we use to slice a `(num_secs * fs, n_channels)`-dimensional subset of time series data as `signal`.

In [4]:
# Get sampling frequency (to nearest integer)
fs = round(nwb.acquisition["ElectricalSeries"].rate)

# Get signal, number of channels
signal = nwb.acquisition["ElectricalSeries"].data[start * fs:(start + num_secs) * fs, :]
_, n_channels = signal.shape
signal.shape

(183120, 256)

Each subject takes a single break while continuing to measure ECoG, which is available under the `"epochs"` attribute. This break can be used as an ECoG baseline. We can find out and store when this happens (in seconds) as `start_break` and `stop_break`.

In [5]:
# Get start, stop time for ECoG baseline
start_break = nwb.epochs["start_time"][0]
stop_break =  nwb.epochs["stop_time"][0]
start_break, stop_break

(318.0, 334.0)

Invalid electrodes are also flagged by index under the `"bads"` attribute, which we can store as `bads`.

In [6]:
# Get "bad" electrodes (invalid data)
bads = [str(i) for i, bad in enumerate(nwb.electrodes["bad"][:]) if bad]
bads

['64', '65', '66', '89', '255']

This is enough information to create an [MNE](https://mne.tools/stable/index.html) `Info` object and MNE `Raw` object, which work together to simplify filtering, plotting, and analysis tasks. MNE prefers channel-first ordering so we pass MNE the transpose of `signal`, that is `signal.T`.

In [7]:
# Create MNE info
# Set "bad" electrodes
info = mne.create_info(n_channels, fs, ch_types="ecog", verbose="error")
info["bads"] = bads

# Create Raw object from 
raw = mne.io.RawArray(signal.T, info)

Creating RawArray with float64 data, n_channels=256, n_times=183120
    Range : 0 ... 183119 =      0.000 ...    60.000 secs
Ready.


See the documentation for MNE's `Raw` object [here](https://mne.tools/stable/generated/mne.io.Raw.html) for details, examples, and how to use it.

## Related Links

- [Bouchard 2013 – "Functional organization of human sensorimotor cortex for speech articulation"](https://www.nature.com/articles/nature11911)
- [Bouchard 2013 – Supplementary Material](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3606666/bin/NIHMS436351-supplement-1.pdf)
- [PyNWB docs – "NWB basics"](https://pynwb.readthedocs.io/en/latest/tutorials/general/file.html)

**end**.